<a href="https://colab.research.google.com/github/rameshavinash94/CMPE255_ANN/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IMPORTING LIBRARIES**

In [ ]:
#Have tried multiple libraries
#Download the necessary modules : spacy english language module, google universal sentece encoder, faiss-cpu, datasketch, annoy, nmslib
!python -m spacy download en_core_web_md
!pip install spacy_universal_sentence_encoder
!pip install faiss-cpu
!pip install datasketch
!pip install annoy
!pip install nmslib

In [ ]:
# Importing necessary modules
import pandas as pd
import numpy as np
import time
from datasketch import MinHash, MinHashLSHForest
import re
import faiss
import annoy
import spacy_universal_sentence_encoder
import nmslib
from annoy import AnnoyIndex
import spacy

## **LOADING AND PREPROCESSING DATASET**

In [18]:
#load the dataset
papers = pd.read_csv("/content/papers.csv")

# Print out the head of the dataframe
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [19]:
#check shape
papers.shape

(7241, 7)

In [20]:
#extracting papers that contains only abstract to avoid hugh chunk of processing in colab
abstract_papers = papers[papers['abstract'] != "Abstract Missing"]

In [21]:
#check shape post filtering
abstract_papers.shape

(3924, 7)

In [22]:
# Remove unnecssary columns
final_dataset = abstract_papers.drop(['id', 'event_type', 'pdf_name','paper_text'], axis=1)

# Print out the first few rows of dataframe 
final_dataset.head()

,year,title,abstract
941,2000,Algorithms for Non-negative Matrix Factorization,Non-negative matrix factorization (NMF) has pr...
1067,2001,Characterizing Neural Gain Control using Spike...,Spike-triggered averaging techniques are effec...
2384,2007,Competition Adds Complexity,It is known that determinining whether a DEC-P...
2385,2007,Efficient Principled Learning of Thin Junction...,We present the first truly polynomial algorith...
2388,2007,Regularized Boost for Semi-Supervised Learning,Semi-supervised inductive learning concerns ho...


In [23]:
#lets check the abstract feature 
final_dataset['abstract']

941     Non-negative matrix factorization (NMF) has pr...
1067    Spike-triggered averaging techniques are effec...
2384    It is known that determinining whether a DEC-P...
2385    We present the first truly polynomial algorith...
2388    Semi-supervised inductive learning concerns ho...
                              ...                        
6943    We revisit the classical analysis of generativ...
6944    PAC maximum                                   ...
6945    Community detection, which focuses on clusteri...
6946    We propose a general framework for interactive...
6947    We consider maximum likelihood estimation of l...
Name: abstract, Length: 3924, dtype: object

## **LSH**

In [24]:
#load english language module of spacy
nlp = spacy.load('en_core_web_md')

# get stop words list
stopwords = nlp.Defaults.stop_words

# set permutations
permutations = 128

In [25]:
#lets create shringles based on whitespaces and then remove stop words 
# if required, we can also convert to canonical form using lemmatization, here not doing it.
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    #convert to lower case
    tokens = text.lower()
    #split into tokens
    tokens = tokens.split()
    cleaned=[]
    for x in tokens:
      if not x in (stopwords):      
          cleaned.append('{a}'.format(a=x))
    #return the canonical form
    return ' '.join(cleaned)

In [26]:
def get_forest(final_dataset, perms):
    minhash = []
    for text in final_dataset['abstract']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
    forest = MinHashLSHForest(num_perm=perms)
    for i,m in enumerate(minhash):
        forest.add(i,m)
    forest.index()    
    return forest

In [27]:
def predict(text, database, perms, num_results, forest):
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))    
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    result = database.iloc[idx_array]['title']    
    return result

In [28]:
#get the forest
forest = get_forest(final_dataset, permutations)

In [29]:
# choose the no.of recommendations
num_recommendations = 10

#select the title for which we need recommendations
title = 'Algorithms for Non-negative Matrix Factorization'

print("The Most Similar Papers To: '{title}' are listed below:".format(title=title))

#get the results by running predict
result = predict(title, final_dataset, permutations, num_recommendations, forest)
print("\n")
print("#"*100)
for x,y in enumerate(result):
    print("{x}.) {y}".format(x=x+1,y=y))

The Most Similar Papers To: 'Algorithms for Non-negative Matrix Factorization' are listed below:


####################################################################################################
1.) Diverse Sequential Subset Selection for Supervised Video Summarization
2.) Estimating Spatial Layout of Rooms using Volumetric Reasoning about Objects and Surfaces
3.) An online Hebbian learning rule that performs Independent Component Analysis
4.) Subject independent EEG-based BCI decoding
5.) Premise Selection for Theorem Proving by Deep Graph Embedding
6.) From Deformations to Parts: Motion-based Segmentation of 3D Objects
7.) Facial Expression Transfer with Input-Output Temporal Restricted Boltzmann Machines
8.) Saliency, Scale and Information: Towards a Unifying Theory
9.) Convergence of Monte Carlo Tree Search in Simultaneous Move Games
10.) Solid Harmonic Wavelet Scattering: Predicting Quantum Molecular Energy from Invariant Descriptors of 3D  Electronic Densities


## **Using Google Universal Sentence Encoder to create vectors prior trying out other ANN alogrithms**

In [30]:
# going to use medium English language module of universal sentence encoder using Spacy(Make use of Google's Universal Sentence Encoder directly within spaCy)
nlp = spacy_universal_sentence_encoder.load_model('en_use_md')

Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB



In [32]:
#picking the abstract for all papers and converting it to vector
abstract = final_dataset.iloc[:,2].values
vector_for_abstract_title=[]
for x in abstract:
  vector_for_abstract_title.append(nlp(x).vector)

In [33]:
#creating dictionary with title and abstract vectors
final=dict()
final['title'] = final_dataset.iloc[:,1].values
final['abstract_vector']=np.array(vector_for_abstract_title)

In [34]:
#title stored as numpy array
final['title']

array(['Algorithms for Non-negative Matrix Factorization',
       'Characterizing Neural Gain Control using Spike-triggered Covariance',
       'Competition Adds Complexity', ...,
       'On clustering network-valued data',
       'A General Framework for Robust Interactive Learning',
       'Multi-view Matrix Factorization for Linear Dynamical System Estimation'],
      dtype=object)

In [35]:
#view the shape of vector created by universal sentence encoder
final['abstract_vector'].shape

(3924, 512)

**As it can be seen whether it is a word, sentence or phrase, the sentence encoder is able to give an embedding vector of size 512**

## **Trying out Other ANN Algorithms**

## **Exhaustive Search**

In [36]:
#exaustive search class
class Exhaustive():
    def __init__(self, vectors, labels):
      #get the shape of the vector
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
          #faiss.IndexFlatL2 slices the input vectors in chunks smaller than blocksize_add and calls add_core
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=11):
        distances, indices = self.index.search(vectors, k)
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [37]:
#create object for the class and call the build function
index = Exhaustive(final["abstract_vector"], final['title'])
index.build()

In [38]:
#extract the top 10 similar papers for a topic
paper_abstract = final['abstract_vector'][0:1]
print("The Most Similar Papers To: '{paper_title}' are listed below:".format(paper_title=final['title'][0]))
print("\n")
print("#"*100)
for x,y in enumerate(index.query(paper_abstract)):
  if x!=0:
    print("{x}.) {y}".format(x=x,y=y))

The Most Similar Papers To: 'Algorithms for Non-negative Matrix Factorization' are listed below:


####################################################################################################
1.) Speeding Up Latent Variable Gaussian Graphical Model Estimation via Nonconvex Optimization
2.) Orthogonal NMF through Subspace Exploration
3.) Convergent Temporal-Difference Learning with Arbitrary Smooth Function Approximation
4.) Newton-Stein Method: A Second Order Method for GLMs via Stein's Lemma
5.) On Algorithms for Sparse Multi-factor NMF
6.) Approximation Algorithms for 
7.) A New Alternating Direction Method for Linear Programming
8.) Randomized Block Krylov Methods for Stronger and Faster Approximate Singular Value Decomposition
9.) Stochastic Variance Reduction Methods for Saddle-Point Problems
10.) Convergence and Energy Landscape for Cheeger Cut Clustering


## **Annoy**

In [39]:
#annoy class
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimension)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=11):
        indices = self.index.get_nns_by_vector(
              vector.tolist(), 
              k)                                           
        return [self.labels[i] for i in indices]

In [40]:
#build the annoy index
index = AnnoyIndex(final["abstract_vector"], final["title"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [41]:
#extract the top 10 similar papers for a topic

paper_abstract=index.query(final["abstract_vector"][0])

print("The Most Similar Papers To: '{paper_title}' are listed below:".format(paper_title=paper_abstract[0]))
print("\n")
print("#"*100)
for x,y in enumerate(paper_abstract):
  if x!=0:
    print("{x}.) {y}".format(x=x,y=y))

The Most Similar Papers To: 'Algorithms for Non-negative Matrix Factorization' are listed below:


####################################################################################################
1.) Speeding Up Latent Variable Gaussian Graphical Model Estimation via Nonconvex Optimization
2.) Orthogonal NMF through Subspace Exploration
3.) Convergent Temporal-Difference Learning with Arbitrary Smooth Function Approximation
4.) On Algorithms for Sparse Multi-factor NMF
5.) Approximation Algorithms for 
6.) A New Alternating Direction Method for Linear Programming
7.) Randomized Block Krylov Methods for Stronger and Faster Approximate Singular Value Decomposition
8.) Stochastic Variance Reduction Methods for Saddle-Point Problems
9.) Convergence and Energy Landscape for Cheeger Cut Clustering
10.) Sum-of-Squares Lower Bounds for Sparse PCA


## **HNSW**

In [42]:
#hnsw class created using using nmslib
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=11):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [43]:
#built the grpah
index = NMSLIBIndex(final["abstract_vector"], final['title'])
index.build()

In [44]:
#extract the top 10 similar papers for a topic

paper_abstract=index.query(final["abstract_vector"][0])

print("The Most Similar Papers To: '{paper_title}' are listed below:".format(paper_title=paper_abstract[0]))
print("\n")
print("#"*100)
for x,y in enumerate(paper_abstract):
  if x!=0:
    print("{x}.) {y}".format(x=x,y=y))

The Most Similar Papers To: 'Algorithms for Non-negative Matrix Factorization' are listed below:


####################################################################################################
1.) Speeding Up Latent Variable Gaussian Graphical Model Estimation via Nonconvex Optimization
2.) Orthogonal NMF through Subspace Exploration
3.) Convergent Temporal-Difference Learning with Arbitrary Smooth Function Approximation
4.) Newton-Stein Method: A Second Order Method for GLMs via Stein's Lemma
5.) On Algorithms for Sparse Multi-factor NMF
6.) Approximation Algorithms for 
7.) A New Alternating Direction Method for Linear Programming
8.) Randomized Block Krylov Methods for Stronger and Faster Approximate Singular Value Decomposition
9.) Stochastic Variance Reduction Methods for Saddle-Point Problems
10.) Convergence and Energy Landscape for Cheeger Cut Clustering


## **Product Quantization**

In [45]:
#product quantization class using faiss
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [46]:
#build the index
index = IVPQIndex(final["abstract_vector"], final['title'])
index.build()

In [47]:
#extract the top 10 similar papers for a topic
paper_abstract = final['abstract_vector'][0:1]
print("The Most Similar Papers To: '{paper_title}' are listed below:".format(paper_title=final['title'][0]))
print("\n")
print("#"*100)
for x,y in enumerate(index.query(paper_abstract)):
  if x!=0:
    print("{x}.) {y}".format(x=x,y=y))

The Most Similar Papers To: 'Algorithms for Non-negative Matrix Factorization' are listed below:


####################################################################################################
1.) On Learning Rotations
2.) A Non-convex One-Pass Framework for Generalized Factorization Machine and Rank-One Matrix Sensing
3.) Scalable Methods for Nonnegative Matrix Factorizations of Near-separable Tall-and-skinny Matrices
4.) Probabilistic Low-Rank Matrix Completion with Adaptive Spectral Regularization Algorithms
5.) On Algorithms for Sparse Multi-factor NMF
6.) CMA-ES with Optimal Covariance Update and Storage Complexity
7.) Mistake Bounds for Binary Matrix Completion
8.) Deterministic Symmetric Positive Semidefinite Matrix Completion
9.) Algorithms for Non-negative Matrix Factorization
